In [3]:
import os
import sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.optim as optim
import math
from tensorboardX import SummaryWriter

import import_ipynb
from CustomDataset import ControlsDataset
from Model import ConvNet

REPORT_EVERY_ITER = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"
print('using device', device)

importing Jupyter notebook from Model.ipynb
   Number  Angle
0       0      9
1       1      9
2       2      9
3       3      9
4       4      9
5       5      9
6       6      9
7       7      9
8       8      9
9       9      9
units after conv 512
conv parameters:  168224
fc parameters:  166804
input torch.Size([256, 3, 480, 640])
tensor([[ 1.6291, -0.4316,  0.9552,  ...,  0.2999,  1.8533,  0.4294],
        [ 1.8139, -0.9876,  1.0120,  ...,  0.1016,  1.9854,  0.2627],
        [ 1.8472, -0.6828,  1.0593,  ...,  0.1325,  1.7137,  0.4267],
        ...,
        [ 1.6707, -0.6002,  1.2496,  ...,  0.1934,  1.6989,  0.3790],
        [ 1.8006, -0.9413,  1.1329,  ...,  0.0251,  1.9035,  0.3072],
        [ 1.6474, -0.6626,  1.4250,  ...,  0.2373,  2.5900,  0.1302]],
       grad_fn=<AddmmBackward>)
output torch.Size([256, 20])
using device cpu


In [4]:
dataset = ControlsDataset()
dataset.convertTOClass()
print("Stack size", dataset.stack_size)
# dataloader = DataLoader(dataset, batch_size = 256, shuffle = True, num_workers = 0)
net = ConvNet().to(device)

print("Data size", dataset.data_frame.size)
print("number of parameters: ", sum(p.numel() for p in net.parameters()))

Stack size 1
units after conv 512
conv parameters:  168224
fc parameters:  166804
Data size 22038
number of parameters:  335028


In [5]:
torch.cuda.empty_cache()

In [6]:
print(dataset.data_frame.head(10))

   Number  Angle
0       0      9
1       1      9
2       2      9
3       3      9
4       4      9
5       5      9
6       6      9
7       7      9
8       8      9
9       9      9


In [7]:
validation_split = .2
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

# spliting the dataset
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


# Training data loader # NOTE had to remove shuffle
dataloader = DataLoader(dataset, batch_size = 256, num_workers = 0, sampler=train_sampler)

# Validation data loader # NOTE had to remove shuffle
validLoader = DataLoader(dataset, batch_size = 256, num_workers = 0, sampler=valid_sampler)

print("Total training stacks", len(dataloader))
print("Total validation stacks",len(validLoader))

Total training stacks 35
Total validation stacks 9


In [8]:
epochs = 25
optimizer = optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss() # Changed from Mean-Squared to Cross-Entropy
writer = SummaryWriter()
iter_no = 0

torch.cuda.empty_cache()
for epoch in range(epochs):
    for i_batch, sampled_batch in enumerate(dataloader):
        iter_no += 1
        images = sampled_batch['image'].to(device).float()
        controls = sampled_batch['control'].to(device).long()
        print("imagesSize: ", images.size(), "controlsSize:", controls.size())
        optimizer.zero_grad()
        prediction = net(images)
        
        controls = torch.flatten(controls)
        print(prediction.size(), controls.size())
        loss = criterion(prediction, controls)
        loss.backward()
        optimizer.step()

        if iter_no % REPORT_EVERY_ITER == 0:
            writer.add_scalar("Loss", loss.item(), iter_no)
            torch.save(net.state_dict(), "snapshots/saved_model_{}".format(loss.item()))
        
        out = "{0},{1}\tLoss:{2}\tAllocated:{3}GB\tCached:{4}GB\n"
        print(out.format(str(epoch),
                        str(iter_no),
                        round(loss.item(),5),
                         'na', 'na'
#                         round(torch.cuda.memory_allocated(0)/1024**3,3),
#                         round(torch.cuda.memory_allocated(0)/1024**3,3)
                        ))
        torch.cuda.empty_cache()

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-1.3201, -1.0427,  0.6101,  ...,  1.7291,  0.8079,  1.9942],
        [-0.9376, -0.9584,  0.0901,  ...,  1.9816,  0.9418,  1.4599],
        [-1.0834, -0.5917,  0.6566,  ...,  1.7480,  1.1641,  1.2342],
        ...,
        [-1.2901, -0.7337,  0.0058,  ...,  2.0439,  0.4704,  1.2447],
        [-1.8945, -0.7674,  1.0277,  ...,  2.1893,  0.9297,  2.0602],
        [-1.1960,  0.1260,  0.9092,  ...,  2.4461,  0.9637,  2.0132]],
       grad_fn=<AddmmBackward>)
torch.Size([256, 20]) torch.Size([256])
0,1	Loss:4.18003	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-1.0009, -0.7573,  0.0441,  ...,  1.1721,  0.8049,  1.3038],
        [-0.5639, -0.3504,  0.1465,  ...,  0.8900,  0.5261,  1.0396],
        [-0.9787, -0.3626,  0.1129,  ...,  1.0658,  0.6778,  1.1843],
        ...,
        [-1.2825, -0.4866,  0.1521,  ...,  1.9180,  0.4597,  1.4658]

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-1.9424, -0.8848,  0.1731,  ..., -0.2963, -0.2585, -0.1502],
        [-1.9990, -1.2782, -0.2111,  ..., -0.1948, -0.2326, -0.3675],
        [-2.3294, -0.8387,  0.0950,  ..., -0.2834, -0.1265, -0.1540],
        ...,
        [-1.8545, -0.9501,  0.0690,  ..., -0.0837, -0.3067, -0.0402],
        [-2.1890, -1.0995,  0.1068,  ...,  0.0164, -0.3966, -0.1311],
        [-1.6993, -0.7759,  0.0802,  ..., -0.0558, -0.1134, -0.2067]],
       grad_fn=<AddmmBackward>)
torch.Size([256, 20]) torch.Size([256])
0,14	Loss:1.71105	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-1.8760, -0.8887, -0.0099,  ..., -0.0468, -0.2593, -0.1414],
        [-2.2995, -1.0018, -0.2224,  ..., -0.0068, -0.4063, -0.1236],
        [-2.0286, -0.8534,  0.1415,  ..., -0.1741, -0.1140, -0.1460],
        ...,
        [-1.7828, -0.7373, -0.2714,  ...,  0.2036, -0.4427,  0.1168

0,26	Loss:1.50029	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-1.9591, -0.6253, -0.2598,  ...,  0.0875, -0.4179,  0.0678],
        [-2.0714, -0.9434, -0.2955,  ...,  0.0877, -0.4835, -0.0627],
        [-2.2198, -0.6796, -0.6398,  ...,  0.2736, -0.2071,  0.0881],
        ...,
        [-1.5047, -0.4800, -0.2253,  ...,  0.0954, -0.2344,  0.1083],
        [-1.5642, -0.4675, -0.0445,  ...,  0.2860, -0.2869,  0.0707],
        [-2.3179, -0.7040, -0.5150,  ...,  0.2763, -0.2967,  0.0470]],
       grad_fn=<AddmmBackward>)
torch.Size([256, 20]) torch.Size([256])
0,27	Loss:1.68	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-1.8439e+00, -6.4005e-01, -2.2125e-01,  ...,  1.5164e-01,
         -2.5174e-01,  3.7201e-02],
        [-1.6203e+00, -4.9709e-01, -1.6756e-01,  ...,  6.4422e-02,
         -2.2167e-01,  8.6003e-02],
        [-1.8324e+00, -6.2186e-01, -1.446

1,39	Loss:1.54487	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-1.6450, -0.7447,  0.0374,  ..., -0.5075, -0.4446,  0.0113],
        [-1.3570, -0.6363, -0.2921,  ..., -0.6095, -0.4753, -0.0735],
        [-1.9621, -0.9898, -0.0571,  ..., -0.3438, -0.7702, -0.1922],
        ...,
        [-1.6673, -0.6794, -0.3142,  ..., -0.4941, -0.8041,  0.0676],
        [-2.1399, -0.9171,  0.0235,  ..., -0.7098, -0.6324,  0.0966],
        [-1.5166, -0.5654,  0.2479,  ..., -0.5106, -0.5417,  0.1218]],
       grad_fn=<AddmmBackward>)
torch.Size([256, 20]) torch.Size([256])
1,40	Loss:1.38809	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-2.1562, -0.8715, -0.7179,  ..., -0.5748, -0.7535, -0.4001],
        [-2.2241, -1.3391, -0.4482,  ..., -0.7447, -1.1837, -0.2162],
        [-2.2126, -0.6629, -0.3417,  ..., -0.5634, -0.8505, -0.2540],
        ...,
        [-1.5441, -0.

1,52	Loss:1.39393	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-2.0273, -0.8977, -0.4739,  ..., -0.4925, -1.0909, -0.0822],
        [-1.8751, -0.7146, -0.6106,  ..., -0.0616, -0.9621, -0.1525],
        [-2.0531, -0.9260, -0.4423,  ..., -0.4465, -1.1348, -0.0759],
        ...,
        [-2.4308, -1.8306, -0.7037,  ..., -0.7588, -1.9278,  0.0424],
        [-2.5542, -1.1892, -1.4750,  ..., -0.4700, -1.5224,  0.0469],
        [-2.0945, -1.3028, -0.8292,  ..., -0.6245, -1.1596,  0.0392]],
       grad_fn=<AddmmBackward>)
torch.Size([256, 20]) torch.Size([256])
1,53	Loss:1.50191	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-1.7940, -1.3135, -0.2290,  ..., -0.4322, -1.4829, -0.0032],
        [-2.2180, -1.2882, -0.9367,  ..., -0.3761, -1.3766, -0.1874],
        [-2.4119, -1.0236, -0.9194,  ..., -0.1854, -1.6568,  0.0686],
        ...,
        [-1.8545, -1.

...

23,815	Loss:0.23085	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[ 1.0959e+00, -2.9047e-01,  9.3089e-02,  ..., -5.7194e+00,
         -7.4130e+00, -1.4443e+01],
        [-9.2942e+00, -2.6010e+00, -1.6787e+00,  ..., -1.4239e-02,
          2.0939e+00, -1.9798e-01],
        [-5.7690e-01, -4.0301e+00, -4.1677e-01,  ..., -4.2468e+00,
         -1.5390e+00,  1.7086e-01],
        ...,
        [-1.7541e+00,  1.6977e+00, -1.1676e+00,  ..., -3.3363e+00,
         -1.8094e+00, -2.4710e+00],
        [-1.4308e-01, -3.0935e-01, -1.3096e+00,  ..., -1.3062e+00,
         -2.4334e+00, -4.4601e+00],
        [-2.4202e+00, -6.0140e-01, -1.2904e+00,  ..., -3.5208e+00,
         -6.7978e+00, -2.3525e+00]], grad_fn=<AddmmBackward>)
torch.Size([256, 20]) torch.Size([256])
23,816	Loss:0.22449	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[-1.0049,  1.8205,  4.2677,  ..., -8.6704

tensor([[-0.2203,  0.5973, -3.3846,  ..., -5.9831, -1.7167, -8.7610],
        [-0.2013, -1.7592,  0.2082,  ..., -0.4543, -4.1980, -3.5832],
        [-2.7191,  1.9293,  2.7738,  ..., -6.1411, -4.3295, -7.3851],
        ...,
        [ 1.5927, -1.1933,  0.5215,  ..., -6.3941, -3.2761, -3.1855],
        [-4.3231,  2.2425,  1.4830,  ..., -3.4888, -2.3000, -6.7106],
        [ 0.7013,  0.2893,  3.5985,  ..., -5.0844, -4.9242, -5.3300]],
       grad_fn=<AddmmBackward>)
torch.Size([256, 20]) torch.Size([256])
23,828	Loss:0.24991	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[  2.9111,  -0.4911,  -2.9880,  ...,  -1.6185,  -5.8577,  -2.8936],
        [  0.6094,   2.6120,   2.0635,  ...,  -5.1792,  -5.8573, -10.1774],
        [ -0.3256,  -3.1672,   1.2981,  ...,  -1.4120,  -0.0269,   5.1511],
        ...,
        [ -4.2247,  -0.2490,  -1.5519,  ...,  -2.8247,  -4.2033, -11.8519],
        [ -3.2631,  -2.7235,   0.4042,  ...,  -1.9

23,840	Loss:0.23602	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[ -7.6410,   0.9081,   2.3015,  ...,  -2.9058,  -2.7804,   3.2553],
        [ -1.2043,  -2.7730,  -0.6438,  ...,  -2.5928,  -2.6708,  -2.7711],
        [ -3.3646,  -5.0735,  -3.0621,  ...,   2.0738,  -3.3621,  -1.2445],
        ...,
        [ -0.6886,   2.6577,   4.8764,  ...,  -1.6591,  -1.7456,  -3.0218],
        [  2.4930,  -2.4620,  -2.4147,  ...,  -3.2914,  -4.7507, -12.1533],
        [ -2.2250,   1.3429,   4.9236,  ...,  -1.5387,  -0.6908,  -8.0701]],
       grad_fn=<AddmmBackward>)
torch.Size([256, 20]) torch.Size([256])
24,841	Loss:0.18659	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[ -0.1414,  -0.0161,  -2.0455,  ...,  -3.2574,  -3.5542,  -4.1972],
        [ -3.5733,  -2.2977,  -1.1891,  ...,   6.9901,   3.1256,  -2.4200],
        [ -1.4170,  -0.5229,  -7.0942,  ...,  -2.049

tensor([[ -3.5059,  -1.0561,   5.0466,  ...,  -3.8389,  -3.1166,  -2.0483],
        [ -0.9425,   1.7624,  -0.1243,  ...,  -3.7029,  -1.6296,  -2.2252],
        [ -0.5934,  -2.2027,   0.2227,  ...,  -2.9921,  -2.3343,  -4.5112],
        ...,
        [ -2.7094,  -1.0315,  -4.7181,  ...,  -3.3118,  -4.7219,  -7.2202],
        [ -1.8619,   2.0888,   4.5597,  ...,  -2.0677,  -1.2041,  -8.8758],
        [  1.5080,   2.3397,   0.8159,  ...,  -2.6810,  -4.5666, -10.4161]],
       grad_fn=<AddmmBackward>)
torch.Size([256, 20]) torch.Size([256])
24,853	Loss:0.20592	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[  0.6331,  -1.8440,   1.0438,  ...,  -1.5349,  -3.4592,  -6.9997],
        [ -4.3179,  -1.5082,  -1.4036,  ...,  -5.6927,  -3.6841, -12.8835],
        [  1.8892,   0.2538,  -1.3224,  ...,  -8.1059,  -1.7854,  -5.7991],
        ...,
        [  3.6973,  -4.0230,  -2.9232,  ...,  -0.9497,  -3.5747,  -6.7058],
        [  1.1

24,865	Loss:0.20609	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[  3.1399,  -0.6419,   4.1570,  ...,  -5.1718,  -3.5008, -12.3602],
        [  0.8838,  -0.2689,  -3.1901,  ...,  -4.5282,  -4.9018,  -9.3290],
        [ -0.5990,   1.2585,   3.2127,  ...,  -2.9913,  -4.5203,  -8.3821],
        ...,
        [ -2.2188,   0.5083,   1.8662,  ...,  -6.1295,  -5.4267,  -5.8705],
        [ -0.1309,   0.7132,  -5.6675,  ...,   1.5287,  -4.3620,  -9.0279],
        [ -1.6144,   0.3544,   0.7053,  ...,  -1.7590,  -0.1771,  -1.0544]],
       grad_fn=<AddmmBackward>)
torch.Size([256, 20]) torch.Size([256])
24,866	Loss:0.27595	Allocated:naGB	Cached:naGB

imagesSize:  torch.Size([256, 3, 480, 640]) controlsSize: torch.Size([256, 1])
tensor([[ 1.9828,  0.9582, -1.0831,  ..., -5.9410, -1.3684, -6.1842],
        [-4.6123, -0.3180,  2.1778,  ..., -0.3070, -5.5771, -4.1759],
        [ 0.4784, -1.5815,  4.1823,  ..., -6.1979, -5.0542, -4.8

In [49]:
# Validation Test
iter_no = 0
total = 0
correct = 0
for i_batch, sampled_batch in enumerate(validLoader):
    iter_no += 1
    images = sampled_batch['image'].to(device).float()
    controls = sampled_batch['control'].to(device).long()
    controls = torch.flatten(controls)
    optimizer.zero_grad()
    prediction = net(images)
    
    maximum = torch.argmax(prediction,axis = 1)
    print(maximum)
    print(controls)
    shared = maximum == controls
    shared = 1 * shared
    correct += int(torch.sum(shared))
    print(correct)
    total += len(controls)
    print(correct/total)

tensor([[ -1.3430,  -0.7843,  -0.9322,  ...,  -2.3487,  -4.8785,  -5.7487],
        [ -7.0056,  -3.8967,  -3.1968,  ...,   4.9140,  -0.0906,  -0.5639],
        [ -2.5804,  -1.4535,  -2.2521,  ...,   0.2093,  -2.2434,   1.6080],
        ...,
        [ -5.5809,   3.9301,  -6.5104,  ...,  -2.5037,  -4.3072, -11.3393],
        [  0.5970,  -0.8343,   0.1480,  ...,  -3.8259,  -3.8487,  -2.7238],
        [ -4.4897,  -0.7815,  -3.4670,  ...,  -3.8485,  -5.1744,  -5.1709]],
       grad_fn=<AddmmBackward>)
tensor([ 8, 11, 14,  8,  9, 10,  8,  9,  9,  9,  9,  8,  9,  9,  9,  9, 10,  7,
         9, 10, 14,  9,  8, 11,  8,  9,  9,  9,  9,  9,  9, 12,  9,  7,  8,  9,
        10, 18,  9,  8,  8,  9, 10,  8,  9,  9,  9,  9,  8,  9,  9,  8, 19,  9,
         9,  9,  8,  2,  9,  9,  6,  9,  8, 10,  9,  9,  9,  9,  8,  9,  8,  9,
         9, 19,  9,  9,  9,  9, 18, 18, 10,  4,  9,  9,  9,  9,  9,  6, 10,  9,
         9,  9,  9,  9, 11, 19, 12,  9,  8,  9,  9,  9,  6,  7, 10,  8,  6,  9,
         9,  7,  9

KeyboardInterrupt: 

In [46]:
print(total)
print(correct)
print("Accuracy: {}".format(correct/total))

2203
1003
Accuracy: 0.45528824330458467


In [ ]:
torch.cuda.empty_cache()

In [ ]:
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024))